# Geographic Visualization using plotly

### Bingyi Li

In [1]:
import pandas as pd
from plotly.offline import init_notebook_mode, plot, iplot

In [2]:
init_notebook_mode(connected=True)

It is interesting to know what countries or regions rank the highest in overall happiness. A choropleth map can be helpful for one to learn this.

## Data Preparation

In [14]:
data_17 = pd.read_csv('./2017.csv')

country_code = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

countries = pd.read_csv('https://gist.githubusercontent.com/tadast/8827699/raw/7255fdfbf292c592b75cf5f7a19c16ea59735f74/countries_codes_and_coordinates.csv')
countries = countries.apply(lambda x : x.str.replace('"',''))
countries[['Latitude (average)','Longitude (average)']] = \
countries[['Latitude (average)','Longitude (average)']].astype(float)

## Choropleth Map

In [15]:
merge_17 = data_17.merge(country_code, how='left', left_on='Country', right_on='COUNTRY')
data = [ dict(
        type = 'choropleth',
        locations = merge_17['CODE'],
        z = merge_17['Happiness.Score'],
        text = merge_17['Country'],
        colorscale = [[0.0, 'rgb(49,54,149)'], [0.2, 'rgb(116,173,209)'],
        [0.4, 'rgb(224,243,248)'], [0.6, 'rgb(253,174,97)'], [0.8, 'rgb(215,48,39)'],
        [1.0, 'rgb(165,0,38)']],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Happiness Score'),
      ) ]

layout = dict(
    title = '2017 World Happiness Choropleth Map',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot( fig, validate=False, filename='d3-world-map' )

According to the choropleth map, one can clearly see that countries in North Europe, North America and Oceania tend to have higher happiness score. Countires in Africa currently have lower happiness score

One possible factor that can influence the overall happiness is GDP. Some people believe that the higher the GDP, the better the ecnomic condition, and thus the better life quality. A symbol map can be used to visualize the relationship between GDP and overall happiness.

## Symbol Map

In [16]:
merge_17_2 = merge_17.merge(countries, how='left', on='Country')
merge_17_2['text'] = merge_17_2['Country'] + '<br>GDP ' \
                    + (merge_17_2['GDP (BILLIONS)']).astype(str)+' billion'

limits = [(0,5),(6,20),(21,50),(21,100),(100,200)]
colors = ["rgb(49,54,149)","rgb(116,173,209)","rgb(224,243,248)","rgb(253,174,97)","rgb(215,48,39)"]
scale = 10
country = []
for i in range(len(limits)):
    lim = limits[i]
    df_sub = merge_17_2[lim[0]:lim[1]]
    c = dict(
        type = 'scattergeo',
        locationmode = 'country names',
        lon = df_sub['Longitude (average)'],
        lat = df_sub['Latitude (average)'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['GDP (BILLIONS)']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    country.append(c)

layout = dict(
        title = '2017 World Happiness',
        showlegend = True,
        geo = dict(
            projection=dict( type = 'Mercator' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )


fig = dict( data=country, layout=layout )
iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In the map above, the color of each bubble stands for the rank of overall happiness, and the size of each bubble stands for the GDP of each country in 2017. According to the symbol map, one can clearly see that higher GDP does not necessarily mean higher life quality or overall happiness. So governments should not only focus on increasing the GDP.